# ADS CREATION V8.4

In [ ]:
import pandas as pd
import numpy as np
from dateutil.parser import parse
from datetime import datetime
import csv
import matplotlib.pyplot as plt
import math
import seaborn as sns
from scipy import stats
plt.style.use('fivethirtyeight')
import warnings
from IPython.display import display_html
warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.0f}'.format

## FILTERING FAC_JOB table

In [ ]:
df_facility_job_his = pd.read_csv('gs://aes-datahub-0001-landing/OMS/IPL/HIS_FACILITY_JOB.csv', sep = ";")

In [ ]:
#Changing the date format
df_facility_job_his["CREATION_DATETIME"] = pd.to_datetime(df_facility_job_his["CREATION_DATETIME"], errors ='coerce')
df_facility_job_his["ENERGIZED_DATETIME"] = pd.to_datetime(df_facility_job_his["ENERGIZED_DATETIME"], errors ='coerce')
df_facility_job_his["ETR_DATETIME"] = pd.to_datetime(df_facility_job_his["ETR_DATETIME"], errors ='coerce')
#creating blue sky flags
df_facility_job_his['BLUE_SKY_FLG'] = ( (df_facility_job_his.TOT_LOSS_POWER_FLG == 'T') & 
                                           ((df_facility_job_his.MAJ_OTG_ID == 0) | (df_facility_job_his.MAJ_OTG_ID.isnull())) &
                                           ((df_facility_job_his.ISOLATED_TO_CUST_FLG == 'F') | (df_facility_job_his.ISOLATED_TO_CUST_FLG.isnull())) &
                                           ((df_facility_job_his.ROUTINE_FLG == 'F') | (df_facility_job_his.ROUTINE_FLG.isnull())) &
                                           ((df_facility_job_his.ENERGIZED_DATETIME - 
                                             df_facility_job_his.CREATION_DATETIME).dt.total_seconds().div(60).round(2) > 5) 
                                          & (df_facility_job_his.CREATION_DATETIME.dt.year > 2002) & (df_facility_job_his.CREATION_DATETIME.dt.year < 2020))
print("Total blue sky Events: ", len(df_facility_job_his[df_facility_job_his.BLUE_SKY_FLG == True]))

#creating storm event flags
df_facility_job_his['STORM_EVENT_FLG'] = ( (df_facility_job_his.TOT_LOSS_POWER_FLG == 'T') & 
                                           ((df_facility_job_his.MAJ_OTG_ID != 0) & (df_facility_job_his.MAJ_OTG_ID.notnull())) &
                                           ((df_facility_job_his.ISOLATED_TO_CUST_FLG == 'F') | (df_facility_job_his.ISOLATED_TO_CUST_FLG.isnull())) &
                                           ((df_facility_job_his.ROUTINE_FLG == 'F') | (df_facility_job_his.ROUTINE_FLG.isnull())) &
                                           ((df_facility_job_his.ENERGIZED_DATETIME -
                                             df_facility_job_his.CREATION_DATETIME).dt.total_seconds().div(60).round(2) > 5) 
                                          & (df_facility_job_his.CREATION_DATETIME.dt.year > 2002) & (df_facility_job_his.CREATION_DATETIME.dt.year < 2020))
print("Total Storm Events: ", len(df_facility_job_his[df_facility_job_his.STORM_EVENT_FLG == True]))


#filtering data
occur_remov = [30003001, 33003301, 33003302, 34003400, 34003401, 34003402, 34003403, 34003404, 34003405, 34003406, 34003407, 34003408, 34003409, 35003500,
                35003501, 35003502, 35003503, 35003504, 35003505, 35003506, 35003507, 35003508, 36003600, 36003601, 36003602, 36003603, 36003604, 36003605,
                36003606, 36003607, 36003608, 37003703, 38003802, 38003803, 38003804, 38003807, 39003910, 41004100, 41004101, 41004102, 48004800, 48004802,
                48004803, 49004900, 49004901, 49004902, 50005000, 50005001, 50005002, 52005200, 52005201, 52005202, 52005203, 52005204, 52005205, 52005206,
                52005207, 53005300, 53005301, 53005302, 53005303, 53005304, 53005305, 53005306, 53005307, 53005308, 53005309, 53005310, 54005400, 54005401,
                54005402, 54005403, 54005404, 54005405, 34003410, 30003000, 36503650, 36503651, 36503652, 36503653, 36503654, 36503655, 36503656, 36503657,
                36503658]

init_rows = len(df_facility_job_his)
init_blue_sky = len(df_facility_job_his[df_facility_job_his.BLUE_SKY_FLG == True])
init_storm = len(df_facility_job_his[df_facility_job_his.STORM_EVENT_FLG == True])
init_incident = df_facility_job_his.INCIDENT_ID.nunique()
_incident_ = len(df_facility_job_his[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Initial Rows", init_rows)
print("Initial blue sky events", init_blue_sky)
print("Initial Storm events", init_storm)
print("Number of incident id", init_incident)
print("Unique incident-structure no",_incident_)

print("-------------After Filtering Creation_datetime from 2007 to 2019---------------------")
df_facility_job_his = df_facility_job_his[(df_facility_job_his.CREATION_DATETIME.dt.year > 2006) & (df_facility_job_his.CREATION_DATETIME.dt.year < 2020)]
_incident_ = len(df_facility_job_his[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_facility_job_his))
print("blue sky events", len(df_facility_job_his[df_facility_job_his.BLUE_SKY_FLG == True]))
print("Storm events", len(df_facility_job_his[df_facility_job_his.STORM_EVENT_FLG == True]))
print("Number of incident id", df_facility_job_his.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)

print("-------------After removing CUST_QTY and DOWNSTREAM_CUST_QTY>0----------------------")
df_fac_f1 = df_facility_job_his[(df_facility_job_his.CUST_QTY > 0) | (df_facility_job_his.DOWNSTREAM_CUST_QTY > 0)]
_incident_ = len(df_fac_f1[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_fac_f1))
print("blue sky events", len(df_fac_f1[df_fac_f1.BLUE_SKY_FLG == True]))
print("Storm events", len(df_fac_f1[df_fac_f1.STORM_EVENT_FLG == True]))
print("Number of incident id", df_fac_f1.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)

print("-------------After removing Equip_stn_No as <NCC> and null----------------------")
df_fac_f2 = df_fac_f1[(df_fac_f1.EQUIP_STN_NO != '<NCC>')]
df_fac_f2 = df_fac_f2[~df_fac_f2.EQUIP_STN_NO.isnull()]
_incident_ = len(df_fac_f2[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_fac_f2))
print("blue sky events", len(df_fac_f2[df_fac_f2.BLUE_SKY_FLG == True]))
print("Storm events", len(df_fac_f2[df_fac_f2.STORM_EVENT_FLG == True]))
print("Number of incident id", df_fac_f2.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)

print("-------------After removing Occurence code starting with Cancel, Found OK and duplicate ----------------------")
df_fac_f3 = df_fac_f2[(df_fac_f2.OCCURN_DESC.str.find("CANCEL") == -1)]
df_fac_f3 = df_fac_f3[(df_fac_f3.OCCURN_DESC.str.find("FOUND OK") == -1)]
_incident_ = len(df_fac_f3[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_fac_f3))
print("blue sky events", len(df_fac_f3[df_fac_f3.BLUE_SKY_FLG == True]))
print("Storm events", len(df_fac_f3[df_fac_f3.STORM_EVENT_FLG == True]))
print("Number of incident id", df_fac_f3.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)

print("-------------After removing NaN from DNI, Circuit ID 0 and NaN  from Structure Nos.----------------------")
df_fac_f4 = df_fac_f3[(df_fac_f3.CIRCT_ID != 0)]
df_fac_f4 = df_fac_f4[~df_fac_f4.CIRCT_ID.isnull()]
df_fac_f4 = df_fac_f4[~df_fac_f4.STRCTUR_NO.isnull()]
df_fac_f4 = df_fac_f4[~df_fac_f4.DNI_EQUIP_TYPE.isnull()]
_incident_ = len(df_fac_f4[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_fac_f4))
print("blue sky events", len(df_fac_f4[df_fac_f4.BLUE_SKY_FLG == True]))
print("Storm events", len(df_fac_f4[df_fac_f4.STORM_EVENT_FLG == True]))
print("Number of incident id", df_fac_f4.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)

print("-------------After removing NaN Energized datetime ----------------------")
df_fac_f5 = df_fac_f4[~(df_fac_f4.ENERGIZED_DATETIME.isnull())]
_incident_ = len(df_fac_f5[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_fac_f5))
print("blue sky events", len(df_fac_f5[df_fac_f5.BLUE_SKY_FLG == True]))
print("Storm events", len(df_fac_f5[df_fac_f5.STORM_EVENT_FLG == True]))
print("Number of incident id", df_fac_f5.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)

print("-------------After removing Energized datetime < CREATION_DATETIME----------------------")
df_fac_f6 = df_fac_f5[df_fac_f5.ENERGIZED_DATETIME > df_fac_f5.CREATION_DATETIME]
_incident_ = len(df_fac_f6[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_fac_f6))
print("blue sky events", len(df_fac_f6[df_fac_f6.BLUE_SKY_FLG == True]))
print("Storm events", len(df_fac_f6[df_fac_f6.STORM_EVENT_FLG == True]))
print("Number of incident id", df_fac_f6.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)
                       
print("-------------After removing CLUE_CD not starting with zero----------------------")
df_fac_f7 = df_fac_f6[(df_fac_f6.CLUE_CD.str[:1] == '0') & (df_fac_f6.CLUE_CD.str[:2] != '00')]
df_fac_f7 = df_fac_f7[df_fac_f7.CLUE_CD != '01']
_incident_ = len(df_fac_f7[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_fac_f7))
print("blue sky events", len(df_fac_f7[df_fac_f7.BLUE_SKY_FLG == True]))
print("Storm events", len(df_fac_f7[df_fac_f7.STORM_EVENT_FLG == True]))
print("Number of incident id", df_fac_f7.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)

print("-------------After removing Occurence code starting with Cancel, Found OK and duplicate ----------------------")
df_fac_f8 = df_fac_f7[~(df_fac_f7.OCCURN_CD.isin(occur_remov))]
_incident_ = len(df_fac_f8[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_fac_f8))
print("blue sky events", len(df_fac_f8[df_fac_f8.BLUE_SKY_FLG == True]))
print("Storm events", len(df_fac_f8[df_fac_f8.STORM_EVENT_FLG == True]))
print("Number of incident id", df_fac_f8.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)

# Incident_id to be removed 1750 cases

exclude_inc_ = pd.read_csv(r'EXCLUDE_INC.csv', header=None)[1]

df_fac_final = df_fac_f8[~(df_fac_f8.INCIDENT_ID.isin(exclude_inc_))]

#Changing Flags to 0 and 1 
df_fac_final.BLUE_SKY_FLG = df_fac_final.BLUE_SKY_FLG.apply(lambda x: 1 if x==True else 0) 
df_fac_final.STORM_EVENT_FLG = df_fac_final.STORM_EVENT_FLG.apply(lambda x: 1 if x==True else 0) 

del df_facility_job_his, df_fac_f1, df_fac_f2, df_fac_f3, df_fac_f4, df_fac_f5, df_fac_f6, df_fac_f7

Total blue sky Events:  135199
Total Storm Events:  36691
Initial Rows 1225564
Initial blue sky events 135199
Initial Storm events 36691
Number of incident id 877496
Unique incident-structure no 982110
-------------After Filtering Creation_datetime from 2007 to 2019---------------------
Rows 933167
blue sky events 108457
Storm events 31343
Number of incident id 828102
Unique structure no 868450
-------------After removing CUST_QTY and DOWNSTREAM_CUST_QTY>0----------------------
Rows 625349
blue sky events 97961
Storm events 31096
Number of incident id 538513
Unique structure no 575452
-------------After removing Equip_stn_No as <NCC> and null----------------------
Rows 608729
blue sky events 97656
Storm events 31038
Number of incident id 531561
Unique structure no 568219
-------------After removing Occurence code starting with Cancel, Found OK and duplicate ----------------------
Rows 417173
blue sky events 97499
Storm events 30984
Number of incident id 371570
Unique structure no 38728

In [ ]:
print("Rows", len(df_fac_final))
_incident_ = len(df_fac_final[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("blue sky events", len(df_fac_final[df_fac_final.BLUE_SKY_FLG == 1]))
print("Storm events", len(df_fac_final[df_fac_final.STORM_EVENT_FLG == 1]))
print("Number of incident id", df_fac_final.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)

Rows 132342
blue sky events 76446
Storm events 26284
Number of incident id 101069
Unique structure no 111088


In [ ]:
df_maj_otg_df = df_fac_final[(~df_fac_final.MAJ_OTG_ID.isnull()) & (df_fac_final.MAJ_OTG_ID != 0)]
maj_otg_df = df_maj_otg_df.groupby(['INCIDENT_ID', 'STRCTUR_NO'], as_index = False).agg({'MAJ_OTG_ID' : pd.Series.nunique})
inc_ = maj_otg_df[maj_otg_df.MAJ_OTG_ID>1].INCIDENT_ID

In [ ]:
df_fac_final[df_fac_final.INCIDENT_ID.isin(inc_)]

,FAC_JOB_ID,CIRCT_ID,MAJ_OTG_ID,EQUIP_STN_NO,DIST_NO,HOST_SEQ_ID,PRIORITY_VAL,CUST_QTY,CLUE_CD,CLUE_DESC,CREATION_DATETIME,CALL_QTY,KEY_CUST_QTY,SPLIT_FAC_JOB_FLG,CAUSE_CD,CAUSE_DESC,OCCURN_CD,OCCURN_DESC,CLIMATIC_CD,CLIMATIC_DESC,CITY_NAM,LOC_DESC,WRK_ORD_NUM,COMMENT_TEXT,CALL_ID,KVA_VAL,BOOK_NO,ADDRESS,CIRCT_NAM,CLUE_CD2,INSERTED_DATE,DOWNSTREAM_KVA_VAL,DOWNSTREAM_CUST_QTY,COMPL_DATETIME,TOT_LOSS_POWER_FLG,ISOLATED_TO_CUST_FLG,PLANNED_OUTAGE_FLG,ROUTINE_FLG,DNI_EQUIP_TYPE,SUBST_ID,WORK_ORD_1_NO,WORK_ORD_2_NO,WORK_ORD_3_NO,WORK_ORD_4_NO,WORK_ORD_5_NO,ENERGIZED_DATETIME,DISPLAY_TEXT,POLICE_OPERATOR_ID,POLICE_INC_NO,FIRE_OPERATOR_ID,FIRE_INC_NO,CAD_ID,STRCTUR_NO,FAC_JOB_PARENT_ID,MAJ_INCIDENT_FLG,MAJ_INCIDENT_CAUSE,ZONE_DESC,DIST_DESC,ZONE_ID,GEO_DIST_NO,ETR_DATETIME,SUBST_SHUTDOWN_FLG,HIS_FAC_JOB_COMMENT,CIRCT_DESC,SUBST_DESC,DCNID_VAL,CREW_REMARKS,INCIDENT_ID,CLUE_DESC2,DIST_ABBREV,BLUE_SKY_FLG,STORM_EVENT_FLG


In [ ]:
maj_otg_df = df_maj_otg_df.groupby(['INCIDENT_ID', 'STRCTUR_NO'], as_index = False).agg({'MAJ_OTG_ID' : 'max'})

In [ ]:
df_event_flg = pd.read_csv("gs://aes-datahub-0001-raw/OMS/IPL/ads_event_updated.csv", sep = ";")[['INCIDENT_ID','STRCTUR_NO','CIRCT_ID' ,'DNI_EQUIP_TYPE', 'EVENT']]
len(df_event_flg)

111163

In [ ]:
df_check = df_fac_final.groupby(['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE']).nunique()
df_check.sum()

FAC_JOB_ID              132342
CIRCT_ID                111163
MAJ_OTG_ID               93055
EQUIP_STN_NO            132242
DIST_NO                 111163
HOST_SEQ_ID                  0
PRIORITY_VAL            113987
CUST_QTY                129556
CLUE_CD                 118183
CLUE_DESC               118183
CREATION_DATETIME       111163
CALL_QTY                125699
KEY_CUST_QTY            118875
SPLIT_FAC_JOB_FLG        41461
CAUSE_CD                106960
CAUSE_DESC              106928
OCCURN_CD               111162
OCCURN_DESC             111343
CLIMATIC_CD              89765
CLIMATIC_DESC            89765
CITY_NAM                109724
LOC_DESC                108482
WRK_ORD_NUM                  0
COMMENT_TEXT             53255
CALL_ID                 132342
KVA_VAL                 127327
BOOK_NO                      0
ADDRESS                 110924
CIRCT_NAM               111163
CLUE_CD2                  1918
INSERTED_DATE           111795
DOWNSTREAM_KVA_VAL      127988
DOWNSTRE

In [ ]:
df_numerical = df_fac_final.groupby(['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE' ], as_index = False).agg({'CUST_QTY':'sum','CALL_QTY':'sum','KEY_CUST_QTY':'sum','DOWNSTREAM_CUST_QTY':'sum','KVA_VAL':'mean',
                                       'DOWNSTREAM_KVA_VAL':'mean', 'FAC_JOB_ID': 'max', 'ETR_DATETIME': 'max', 'CREATION_DATETIME': 'min', 'MAJ_OTG_ID' : 'sum',
                                      'ENERGIZED_DATETIME': 'max', 'SUBST_ID': 'min'})

In [ ]:
df_numerical = pd.merge(df_event_flg, df_numerical, on = ['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE' ])

In [ ]:
df_numerical.shape

(111163, 17)

## Adding extra columns like Day Night flag and TTR

In [ ]:
#creating day night flag for outages
df_numerical['DAY_FLAG'] = df_numerical.CREATION_DATETIME.dt.hour.apply(lambda x: 1 if ((x >= 6) & (x<18)) else 0)
df_numerical['TTR'] = (df_numerical.ENERGIZED_DATETIME - df_numerical.CREATION_DATETIME).dt.total_seconds().div(60).round(4)

In [ ]:
df_numerical.shape

(111163, 19)

### CLUE CODE CLEAN

In [ ]:
clue = pd.read_csv('gs://aes-datahub-0001-raw/OMS/IPL/CLUE_pp.csv', sep = ";")
clue = clue[['CLUE_CD','PRIORITY_VAL']]

In [ ]:
clue = clue[clue.CLUE_CD.isin(df_fac_final.CLUE_CD.unique())]
df_fac_final = df_fac_final.drop(columns=['PRIORITY_VAL'])
df_fac_final = pd.merge(df_fac_final, clue, on = ['CLUE_CD'], how = "left")

In [ ]:
df_fac_final = pd.get_dummies(df_fac_final, columns = ['PRIORITY_VAL'])
df_fac_final.CITY_NAM = df_fac_final.CITY_NAM.apply(lambda x: 'INDIANAPOLIS' if(str(x).find('INDIAN') != -1) else x)
df_fac_final.CITY_NAM = df_fac_final.CITY_NAM.apply(lambda x: 'NO_CITY' if(x != x) else x)

In [ ]:
clue['PRIORITY_VAL'].unique()

array([2, 5, 3, 1])

In [ ]:
#segregation of clue code desc
df_fac_final['POLE_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('pole') != -1) else 0)
df_fac_final['PART_LIGHT_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('part lights') != -1) else 0)
df_fac_final['EMERGENCY_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('emergency') != -1) else 0)
df_fac_final['POWER_OUT_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('power out') != -1) else 0)
df_fac_final['TREE_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('tree') != -1) else 0)
df_fac_final['WIRE_DOWN_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('wire down') != -1) else 0)
df_fac_final['IVR_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('ivr') != -1) else 0)
df_fac_final['EQUIPMENT_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.find('EQUIPMENT') != -1) else 0)
df_fac_final['TRANSFORMER_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.find('TRANSFORMER') != -1) else 0)
df_fac_final['OPEN_DEVICE_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.find('OPEN DEVICE') != -1) else 0)


#segration of cause desc
df_fac_final['CAUSE_DESC1'] = df_fac_final[['CAUSE_DESC']].fillna('0')
df_fac_final['OH_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if((x.find('OH') != -1) | (x.find('O.H.') != -1)) else 0)
df_fac_final['UG_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if((x.find('UG') != -1) | (x.find('U.G.') != -1)) else 0)
df_fac_final['ANIMAL_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('ANIMAL') != -1) else 0)
df_fac_final['WEATHER_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('WEATHER') != -1) else 0)
df_fac_final['WEATHER_COLD_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('COLD') != -1) else 0)
df_fac_final['WEATHER_LIGHTNING_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('LIGHTNING') != -1) else 0)
df_fac_final['WEATHER__SNOW_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('SNOW') != -1) else 0)
df_fac_final['WEATHER__WIND_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('WIND') != -1) else 0)
df_fac_final['WEATHER__HEAT_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('HEAT') != -1) else 0)
df_fac_final['WEATHER__FLOOD_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('FLOOD') != -1) else 0)
df_fac_final['PUBLIC_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('PUBLIC') != -1) else 0)
df_fac_final['STREET_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('ST ') != -1) else 0)
df_fac_final['SUBSTATION_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('SUBSTATION') != -1) else 0)
df_fac_final['TREE_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('TREE') != -1) else 0)
df_fac_final['MISCELLANEOUS_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('MISCELLANEOUS') != -1) else 0)
df_fac_final['CUST_REQUEST_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('CUSTOMER REQUEST') != -1) else 0)
df_fac_final['NO_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('NO CAUSE') != -1) else 0)
df_fac_final['PLANNED_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('PLANNED WORK') != -1) else 0)
df_fac_final['NO_OUTAGE_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('NO OUTAGE') != -1) else 0)


#segration of OCCURN desc
df_fac_final['FUSE_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if((x.find('FUSE') != -1) & (x.find('FUSE NOT') == -1)) else 0)
df_fac_final['CUST_EQUIP_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('CUSTOMER EQUIP') != -1) else 0)
df_fac_final['POLE_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('POLE') != -1) else 0)
df_fac_final['TRANSFORMER_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('TRANSFORMER') != -1) else 0)
df_fac_final['METER_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('METER') != -1) else 0)
df_fac_final['SERVICE_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('SERVICE') != -1) else 0)
df_fac_final['CABLE_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('CABLE') != -1) else 0)
df_fac_final['ST_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('ST') != -1) else 0)
df_fac_final['FIRE_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('FIRE') != -1) else 0)
df_fac_final['FOUND_OPEN_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if((x.find('FOUND OPEN') != -1) & (x.find('NOT FOUND OPEN') == -1)) else 0)
df_fac_final['PUBLIC_SAFETY_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('SAFETY') != -1) else 0)
df_fac_final['WIRE_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('WIRE') != -1) else 0)
df_fac_final['SWITCH_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('SWITCH') != -1) else 0)
df_fac_final['CUTOUT_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('CUTOUT') != -1) else 0)
df_fac_final['REGULATOR_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('REGULATOR') != -1) else 0)
df_fac_final['CAP_BANK_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('CAP BANK') != -1) else 0)
df_fac_final['OH_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('OH') != -1) else 0)
df_fac_final['RECLOSER_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('RECLOSER') != -1) else 0)

df_fac_final = df_fac_final.drop(columns = ['CAUSE_DESC1'])

In [ ]:
df_fac_cat = df_fac_final.groupby(['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE'], as_index = False).agg({'POLE_CLUE_FLG': 'sum', 'PART_LIGHT_CLUE_FLG': 'sum',
                                                                          'EMERGENCY_CLUE_FLG': 'sum','POWER_OUT_CLUE_FLG': 'sum',
                                                                          'TREE_CLUE_FLG': 'sum', 'WIRE_DOWN_CLUE_FLG': 'sum',
                                                                          'OPEN_DEVICE_CLUE_FLG':'sum', 'EQUIPMENT_CLUE_FLG': 'sum',
                                                                          'TRANSFORMER_CLUE_FLG':'sum','IVR_CLUE_FLG': 'sum',
                                                                          'OH_CAUSE_FLG': 'sum', 'UG_CAUSE_FLG': 'sum', 
                                                                          'ANIMAL_CAUSE_FLG': 'sum','WEATHER_CAUSE_FLG': 'sum', 
                                                                          'WEATHER_COLD_CAUSE_FLG': 'sum','PUBLIC_CAUSE_FLG': 'sum',
                                                                         'WEATHER_LIGHTNING_CAUSE_FLG': 'sum', 'WEATHER__SNOW_CAUSE_FLG': 'sum',
                                                                          'WEATHER__WIND_CAUSE_FLG': 'sum','WEATHER__HEAT_CAUSE_FLG': 'sum',
                                                                         'WEATHER__FLOOD_CAUSE_FLG': 'sum', 'STREET_CAUSE_FLG': 'sum',
                                                                        'MISCELLANEOUS_CAUSE_FLG':'sum', 'CUST_REQUEST_CAUSE_FLG': 'sum',
                                                                          'SUBSTATION_CAUSE_FLG': 'sum','TREE_CAUSE_FLG': 'sum',
                                                                          'NO_CAUSE_FLG': 'sum', 'PLANNED_CAUSE_FLG': 'sum',
                                                                          'NO_OUTAGE_CAUSE_FLG': 'sum',
                                                                          'PRIORITY_VAL_1.0' : 'sum', 'PRIORITY_VAL_2.0': 'sum', 
                                                                          'PRIORITY_VAL_3.0': 'sum', 'PRIORITY_VAL_5.0': 'sum',
                                                                          'FUSE_OCCURN_FLG': 'sum', 'CUST_EQUIP_OCCURN_FLG': 'sum',
                                                                          'POLE_OCCURN_FLG': 'sum', 'TRANSFORMER_OCCURN_FLG': 'sum', 
                                                                          'METER_OCCURN_FLG': 'sum', 'SERVICE_OCCURN_FLG': 'sum',
                                                                          'CABLE_OCCURN_FLG': 'sum', 'ST_OCCURN_FLG': 'sum',
                                                                          'FIRE_OCCURN_FLG': 'sum', 'FOUND_OPEN_OCCURN_FLG': 'sum',
                                                                          'PUBLIC_SAFETY_OCCURN_FLG': 'sum', 'WIRE_OCCURN_FLG': 'sum',
                                                                          'SWITCH_OCCURN_FLG': 'sum', 'REGULATOR_OCCURN_FLG': 'sum',
                                                                          'CUTOUT_OCCURN_FLG': 'sum','CAP_BANK_OCCURN_FLG': 'sum',
                                                                          'RECLOSER_OCCURN_FLG': 'sum','OH_OCCURN_FLG': 'sum'
                                                                          })
dummy_col = ['POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG','POWER_OUT_CLUE_FLG','OPEN_DEVICE_CLUE_FLG',
                'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG','IVR_CLUE_FLG','EQUIPMENT_CLUE_FLG','TRANSFORMER_CLUE_FLG',
             'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG','WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG',
             'PUBLIC_CAUSE_FLG','WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG','WEATHER__WIND_CAUSE_FLG',
             'WEATHER__HEAT_CAUSE_FLG','CUST_REQUEST_CAUSE_FLG','WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG',
             'SUBSTATION_CAUSE_FLG','TREE_CAUSE_FLG','MISCELLANEOUS_CAUSE_FLG','NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 
             'NO_OUTAGE_CAUSE_FLG',
             'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 
             'METER_OCCURN_FLG', 'SERVICE_OCCURN_FLG','CABLE_OCCURN_FLG', 'ST_OCCURN_FLG', 'FIRE_OCCURN_FLG', 
             'FOUND_OPEN_OCCURN_FLG','PUBLIC_SAFETY_OCCURN_FLG', 'WIRE_OCCURN_FLG', 'SWITCH_OCCURN_FLG',
             'REGULATOR_OCCURN_FLG', 'CUTOUT_OCCURN_FLG','CAP_BANK_OCCURN_FLG','RECLOSER_OCCURN_FLG','OH_OCCURN_FLG',
             'PRIORITY_VAL_1.0','PRIORITY_VAL_2.0','PRIORITY_VAL_3.0','PRIORITY_VAL_5.0']
for i in dummy_col:
    df_fac_cat[i] =  df_fac_cat[i].apply(lambda x: 1 if x>=1 else 0)

df_fac_cat = df_fac_cat[['INCIDENT_ID','STRCTUR_NO','CIRCT_ID', 'DNI_EQUIP_TYPE',
                         'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG','POWER_OUT_CLUE_FLG','OPEN_DEVICE_CLUE_FLG',
                'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG','IVR_CLUE_FLG','EQUIPMENT_CLUE_FLG','TRANSFORMER_CLUE_FLG',
             'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG','WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG',
             'PUBLIC_CAUSE_FLG','WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG','WEATHER__WIND_CAUSE_FLG',
             'WEATHER__HEAT_CAUSE_FLG','CUST_REQUEST_CAUSE_FLG','WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG',
             'SUBSTATION_CAUSE_FLG','TREE_CAUSE_FLG','MISCELLANEOUS_CAUSE_FLG','NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 
              'NO_OUTAGE_CAUSE_FLG',
             'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 
             'METER_OCCURN_FLG', 'SERVICE_OCCURN_FLG','CABLE_OCCURN_FLG', 'ST_OCCURN_FLG', 'FIRE_OCCURN_FLG', 
             'FOUND_OPEN_OCCURN_FLG','PUBLIC_SAFETY_OCCURN_FLG', 'WIRE_OCCURN_FLG', 'SWITCH_OCCURN_FLG',
             'REGULATOR_OCCURN_FLG', 'CUTOUT_OCCURN_FLG','CAP_BANK_OCCURN_FLG','RECLOSER_OCCURN_FLG','OH_OCCURN_FLG',
             'PRIORITY_VAL_1.0','PRIORITY_VAL_2.0','PRIORITY_VAL_3.0','PRIORITY_VAL_5.0']].drop_duplicates()

QC check if the list retains to 111163

In [ ]:
df_fac_cat.shape

(111163, 55)

### Cause and OCCURN desc clean

In [ ]:
df_fac_final['OCCURN_DESC'] = df_fac_final['OCCURN_DESC'].str.replace(";","")

In [ ]:
df_fac_final['OCCURN_DESC']

0                        FUSE\OPEN, FUSE BLOWN
1                        FUSE\OPEN, FUSE BLOWN
2                        FUSE\OPEN, FUSE BLOWN
3                             POLE\NOT DAMAGED
4         CUSTOMER EQUIP.\CONDUIT BENT OR DOWN
                          ...                 
132337             CONDUCTOR/WIRE\PRIMARY DOWN
132338           OPENED FOR SAFETY\FOR OH CREW
132339         OPENED FOR SAFETY\FOR TREE CREW
132340         OPENED FOR SAFETY\FOR TREE CREW
132341         OPENED FOR SAFETY\FOR TREE CREW
Name: OCCURN_DESC, Length: 132342, dtype: object

In [ ]:
t = df_fac_final.groupby(['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE'], as_index = False).agg({'OCCURN_DESC': pd.Series.nunique,'CAUSE_DESC': pd.Series.nunique})

In [ ]:
t[t.OCCURN_DESC > 1] , t[t.CAUSE_DESC > 1]

(       INCIDENT_ID STRCTUR_NO  CIRCT_ID DNI_EQUIP_TYPE  CAUSE_DESC  OCCURN_DESC
 59013   2000853164   615-A/75      1707          2TBOD           1            2,
        INCIDENT_ID STRCTUR_NO  CIRCT_ID DNI_EQUIP_TYPE  CAUSE_DESC  OCCURN_DESC
 35313   2000595721  369-A/326      2457          1TBOH           2            1
 86598   2001169259  469-B/166      5154           FUSE           2            1)

In [ ]:
df_fac_final[df_fac_final.INCIDENT_ID == t[t.OCCURN_DESC > 1].INCIDENT_ID[59013]].CAD_ID

67910    IPL-102412-000184
67933                  NaN
Name: CAD_ID, dtype: object

In [ ]:
#del row with CAD_ID not present
df_fac_final = df_fac_final.drop(67933)

In [ ]:
df_fac_final[df_fac_final.INCIDENT_ID.isin(t[t.CAUSE_DESC > 1].INCIDENT_ID)]

,FAC_JOB_ID,CIRCT_ID,MAJ_OTG_ID,EQUIP_STN_NO,DIST_NO,HOST_SEQ_ID,CUST_QTY,CLUE_CD,CLUE_DESC,CREATION_DATETIME,CALL_QTY,KEY_CUST_QTY,SPLIT_FAC_JOB_FLG,CAUSE_CD,CAUSE_DESC,OCCURN_CD,OCCURN_DESC,CLIMATIC_CD,CLIMATIC_DESC,CITY_NAM,LOC_DESC,WRK_ORD_NUM,COMMENT_TEXT,CALL_ID,KVA_VAL,BOOK_NO,ADDRESS,CIRCT_NAM,CLUE_CD2,INSERTED_DATE,DOWNSTREAM_KVA_VAL,DOWNSTREAM_CUST_QTY,COMPL_DATETIME,TOT_LOSS_POWER_FLG,ISOLATED_TO_CUST_FLG,PLANNED_OUTAGE_FLG,ROUTINE_FLG,DNI_EQUIP_TYPE,SUBST_ID,WORK_ORD_1_NO,WORK_ORD_2_NO,WORK_ORD_3_NO,WORK_ORD_4_NO,WORK_ORD_5_NO,ENERGIZED_DATETIME,DISPLAY_TEXT,POLICE_OPERATOR_ID,POLICE_INC_NO,FIRE_OPERATOR_ID,FIRE_INC_NO,CAD_ID,STRCTUR_NO,FAC_JOB_PARENT_ID,MAJ_INCIDENT_FLG,MAJ_INCIDENT_CAUSE,ZONE_DESC,DIST_DESC,ZONE_ID,GEO_DIST_NO,ETR_DATETIME,SUBST_SHUTDOWN_FLG,HIS_FAC_JOB_COMMENT,CIRCT_DESC,SUBST_DESC,DCNID_VAL,CREW_REMARKS,INCIDENT_ID,CLUE_DESC2,DIST_ABBREV,BLUE_SKY_FLG,STORM_EVENT_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,PUBLIC_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,CUTOUT_OCCURN_FLG,REGULATOR_OCCURN_FLG,CAP_BANK_OCCURN_FLG,OH_OCCURN_FLG,RECLOSER_OCCURN_FLG
39382,2001573885,2457,0,1742831B_B,1,nan,1,09IV,DO NOT USE - IVR\POWER OUT,2009-08-07 11:42:57,1,0,NaN,14001409,PUBLIC\FIRE,21002108,SERVICE\SERVICE WIRE SHORT,N,Normal - Clear,INDIANAPOLIS,** 5014 HILLSIDE AVE 2200E/N,nan,cust said everyone else came back on but her ;...,2026692708,0,nan,5014 HILLSIDE AVE 2200E/N,CRESTVIEW NO. 7,NaN,2009-08-07T14:52:06,0,1,2009-08-07T13:53:03,F,T,F,F,1TBOH,245,NaN,NaN,nan,nan,nan,2009-08-07 13:53:03,369-A/326,nan,nan,nan,nan,IPL-080709-000205,369-A/326,nan,F,PUBLIC\FIRE,IPL,IPL,1,nan,2009-08-07 15:12:57,NaN,NaN,CRESTVIEW NO. 7,CRESTVIEW,nan,NaN,2000595721,NaN,NaN,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
39383,2001573964,2457,0,1742831B_B,1,nan,4,09OD,DO NOT USE - SYSTEM\OPEN DEVICE,2009-08-07 11:42:57,1,0,NaN,12001201,TREE\ON SECONDARY,21002108,SERVICE\SERVICE WIRE SHORT,NaN,NaN,INDIANAPOLIS,No Address : Loc 2000655770,nan,LOCATION CREATED BY OPENING DEVICE: 369-A/326 ...,2026692944,25,nan,NaN,CRESTVIEW NO. 7,NaN,2009-08-07T14:52:06,25,4,2009-08-07T14:51:49,F,T,F,F,1TBOH,245,NaN,NaN,nan,nan,nan,2009-08-07 13:53:03,369-A/326,nan,nan,nan,nan,NaN,369-A/326,nan,F,PUBLIC\FIRE,IPL,IPL,1,nan,2009-08-07 15:12:57,NaN,NaN,CRESTVIEW NO. 7,CRESTVIEW,nan,NaN,2000595721,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
101794,2002277141,5154,nan,1777021_A,1,nan,4,09GC,ACG GROUPABLE CLUE CODE,2016-09-02 09:47:24,1,0,F,33003301,PUBLIC\VEHICLE ACCIDENT,31003100,"FUSE\OPEN, FUSE BLOWN",NaN,NaN,NO_CITY,W/HILLSIDE AT LANGLY,nan,NaN,2030607709,0,nan,W/HILLSIDE AT LANGLY,ROOSEVELT,NaN,2016-09-05T11:49:07,100,4,NaN,T,NaN,NaN,NaN,FUSE,33,NaN,NaN,nan,nan,nan,2016-09-02 10:41:49,469-B/166,nan,nan,nan,nan,NaN,469-B/166,nan,F,NaN,IPL,NaN,1,nan,NaT,NaN,NaN,ROOSEVELT,MASSACHUSETTS,nan,NaN,2001169259,NaN,NaN,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
101795,2002277142,5154,nan,1777021_C,1,nan,3,09GC,ACG GROUPABLE CLUE CODE,2016-09-02 09:47:24,1,0,F,33003301,PUBLIC\VEHICLE ACCIDENT,31003100,"FUSE\OPEN,

In [ ]:
#del row with CAD_ID not present
df_fac_final = df_fac_final.drop(39383)
# #completion time not present
df_fac_final = df_fac_final.drop(101794)
df_fac_final = df_fac_final.drop(101795)

In [ ]:
df_desc = df_fac_final[['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE','OCCURN_DESC','CAUSE_DESC']].drop_duplicates()
df_desc = df_desc.fillna(0)

In [ ]:
t = df_desc.groupby(['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE'], as_index = False).agg({'CAUSE_DESC': 'count'})

In [ ]:
df_desc[df_desc.INCIDENT_ID.isin(t[t.CAUSE_DESC > 1].INCIDENT_ID)]

,INCIDENT_ID,STRCTUR_NO,CIRCT_ID,DNI_EQUIP_TYPE,OCCURN_DESC,CAUSE_DESC
28314,2000532210,378-B/98,2105,FUSE,FUSE\FUSE BLOWN,0
28315,2000532210,378-B/98,2105,FUSE,FUSE\FUSE BLOWN,ANIMAL\BIRD


In [ ]:
#removing null
df_desc = df_desc.drop(28314)
len(df_desc)

111163

In [ ]:
split_desc = df_desc['OCCURN_DESC'].str.split('\\',expand=True)
df_desc['PRIM_OCCURN_DESC'] = split_desc[0]
split_desc = split_desc[1].str.split(', ',expand=True)
df_desc['SECO_OCCURN_DESC'] = split_desc[0]
df_desc['TERT_OCCURN_DESC'] = split_desc[1] 
df_desc['PRIM_OCCURN_DESC'] = df_desc['PRIM_OCCURN_DESC'].str.strip()
df_desc['SECO_OCCURN_DESC'] = df_desc['SECO_OCCURN_DESC'].str.strip()
df_desc['TERT_OCCURN_DESC'] = df_desc['TERT_OCCURN_DESC'].str.strip()

In [ ]:
split_desc = df_desc['CAUSE_DESC'].str.split('\\',expand=True)
df_desc['PRIM_CAUSE_DESC'] = split_desc[0]
df_desc['SECO_CAUSE_DESC'] = split_desc[1]
df_desc = df_desc.drop(columns = ['CAUSE_DESC','OCCURN_DESC'])

### CITY

In [ ]:
# City treatment
def cat_city_treat(group):
    if(group.CITY_NAM.nunique() > 1):
        x = group[group.CITY_NAM != 'NO_CITY'].CITY_NAM.unique()
        group.CITY_NAM = x[0]
        return group
    else:
        return group
df_treated = df_fac_final[['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'CITY_NAM']]
df_treated = df_treated.groupby(['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE'], as_index = False).apply(cat_city_treat)

In [ ]:
len(df_treated[['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'CITY_NAM']].drop_duplicates())

111163

QC check complete

In [ ]:
df_treated = df_treated[['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE','CITY_NAM']].drop_duplicates()
df_fac_cat = pd.merge(df_fac_cat, df_treated, on = ['INCIDENT_ID','STRCTUR_NO','CIRCT_ID', 'DNI_EQUIP_TYPE'])

In [ ]:
df_fac_cat.shape

(111163, 56)

In [ ]:
df_desc.shape

(111163, 9)

In [ ]:
df_fac_cat  = pd.merge(df_fac_cat, df_desc, on = ['INCIDENT_ID','STRCTUR_NO','CIRCT_ID', 'DNI_EQUIP_TYPE'])

In [ ]:
df_fac_final[df_fac_final['INCIDENT_ID'] == 2000383864].FUSE_OCCURN_FLG

185    1
Name: FUSE_OCCURN_FLG, dtype: int64

In [ ]:
df_fac_cat.shape

(111163, 61)

## X Y Coordinate

In [ ]:
df_his_location = pd.read_csv('gs://aes-datahub-0001-raw/OMS/IPL/HIS_LOCATION_pp.csv', sep = ";")

In [ ]:
df_his_location = df_his_location[['INCIDENT_ID','STRCTUR_NO','GEO_X_COORD','GEO_Y_COORD','PRIMARY_LOC_FLG']].drop_duplicates()
df_his_location = df_his_location[df_his_location.INCIDENT_ID.isin(df_fac_final.INCIDENT_ID)]
df_his_location = df_his_location[df_his_location.STRCTUR_NO.isin(df_fac_final.STRCTUR_NO)]

### changing x y coord to lat and long

In [ ]:
import math
df_his_location['LAT'] = 0
df_his_location['LONG'] = 0
df_his_location = df_his_location.reset_index(drop = True)
for i in range(len(df_his_location)):
    demnorthing = df_his_location.GEO_Y_COORD[i]
    demeasting = df_his_location.GEO_X_COORD[i]
    northing = float(demnorthing) * 0.3048
    easting = float(demeasting) * 0.3048
    om = (northing - 250000 + 4151863.7425) / 6367236.89768
    fo = om + (math.sin(om) * math.cos(om)) * (0.005022893948 + 0.000029370625 * math.pow(math.cos(om), 2) + 0.000000235059 * math.pow(math.cos(om), 4) + 0.000000002181 * math.pow(math.cos(om), 6))
    tf = math.sin(fo) / math.cos(fo)
    nf2 = 0.00673949677548 * math.pow(math.cos(fo), 2)
    rn = 0.9999666667 * 6378137 / math.pow((1 - 0.0066943800229034 * math.pow(math.sin(fo), 2)), 0.5)
    q = (easting - 100000) / rn
    b2 = -0.5 * tf * (1 + nf2)
    b4 = -(1 / 12) * (5 + (3 * math.pow(tf, 2)) + (nf2 * (1 - 9 * math.pow(tf, 2)) - 4 * math.pow(nf2, 2)))
    b6 = (1 / 360) * (61 + (90 * math.pow(tf, 2)) + (45 * math.pow(tf, 4)) + (nf2 * (46 - (252 * math.pow(tf, 2)) - (90 * math.pow(tf, 4)))))
    lat = fo + b2 * math.pow(q, 2) * (1 + math.pow(q, 2) * (b4 + b6 * math.pow(q, 2)))
    b3 = -(1 / 6) * (1 + 2 * math.pow(tf, 2) + nf2)
    b5 = (1 / 120) * (5 + 28 * math.pow(tf, 2) + 24 * math.pow(tf, 4) + nf2 * (6 + 8 * math.pow(tf, 2)))
    b7 = -(1 / 5040) * (61 + 662 * math.pow(tf, 2) + 1320 * math.pow(tf, 4) + 720 * math.pow(tf, 6))
    l = q * (1 + math.pow(q, 2) * (b3 + math.pow(q, 2) * (b5 + b7 * math.pow(q, 2))))
    lon = 1.4951653925 - l / math.cos(fo)
    coord = [(lat * 57.2957795131), (-1 * lon * 57.2957795131)]
    df_his_location['LAT'][i] = str(lat * 57.2957795131)
    df_his_location['LONG'][i] = str(-1 * lon * 57.2957795131)

In [ ]:
(df_his_location['INCIDENT_ID'].astype(str) + df_his_location['STRCTUR_NO'].astype(str)).nunique(), len(df_his_location)

(112054, 112884)

Multiple location, so had to do treating

In [ ]:
def loc_treatment(group):
    group = group.reset_index(drop = True)
    if((group.LAT.nunique() > 1) | (group.LONG.nunique() > 1)):
        x = 0.0
        y = 0.0
        z = 0.0
        for i in range(len(group)):
            latitude = math.radians(float(group.LAT[i]))
            longitude = math.radians(float(group.LONG[i]))
            x += math.cos(latitude) * math.cos(longitude)
            y += math.cos(latitude) * math.sin(longitude)
            z += math.sin(latitude)
        total = len(group)
        x = x / total
        y = y / total
        z = z / total
        central_longitude = math.atan2(y, x)
        central_square_root = math.sqrt(x * x + y * y)
        central_latitude = math.atan2(z, central_square_root)
        group.LAT = math.degrees(central_latitude)
        group.LONG = math.degrees(central_longitude)
        return group
    else:
        return group
df_his_location = df_his_location.groupby(['INCIDENT_ID','STRCTUR_NO'], as_index = False).apply(loc_treatment)

In [ ]:
df_his_location = df_his_location[['INCIDENT_ID','STRCTUR_NO','LAT','LONG']].drop_duplicates()

In [ ]:
(df_his_location['INCIDENT_ID'].astype(str) + df_his_location['STRCTUR_NO'].astype(str)).nunique(), len(df_his_location)

(112054, 112054)

In [ ]:
df_his_location['LAT'].astype(float).min(), df_his_location['LAT'].astype(float).max()

(35.24263651397552, 39.98327607330718)

In [ ]:
df_his_location['LONG'].astype(float).min(), df_his_location['LONG'].astype(float).max()

(-86.7646286032229, -85.9158985730893)

In [ ]:
center_lat = 39.7684
center_long = -86.1581
zone = []
df_his_location = df_his_location.reset_index(drop = True)
for i in range(len(df_his_location)):
    if(float(df_his_location['LAT'][i]) < center_lat):
        if(float(df_his_location['LONG'][i]) < center_long):
            zone.append(1)
        else:
            zone.append(2)
    else:
        if(float(df_his_location['LONG'][i]) < center_long):
            zone.append(4)
        else:
            zone.append(3)
df_his_location['ZONE'] = zone

In [ ]:
df_fac_cat = pd.merge(df_fac_cat, df_his_location, on = ['INCIDENT_ID','STRCTUR_NO'], how = "left")

In [ ]:
df_fac_cat.shape

(111163, 64)

In [ ]:
df_ads = pd.merge(df_numerical, df_fac_cat, on = ['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE'])

In [ ]:
df_ads = df_ads[['FAC_JOB_ID','INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 
         'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DOWNSTREAM_KVA_VAL', 
         'KVA_VAL', 'DAY_FLAG', 'TTR', 'POLE_CLUE_FLG', 
                 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG','POWER_OUT_CLUE_FLG','OPEN_DEVICE_CLUE_FLG',
                'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG','IVR_CLUE_FLG','EQUIPMENT_CLUE_FLG','TRANSFORMER_CLUE_FLG',
             'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG','WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG',
             'PUBLIC_CAUSE_FLG','WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG','WEATHER__WIND_CAUSE_FLG',
             'WEATHER__HEAT_CAUSE_FLG','CUST_REQUEST_CAUSE_FLG','WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG',
             'SUBSTATION_CAUSE_FLG','TREE_CAUSE_FLG','MISCELLANEOUS_CAUSE_FLG','NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 
              'NO_OUTAGE_CAUSE_FLG',
             'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 
             'METER_OCCURN_FLG', 'SERVICE_OCCURN_FLG','CABLE_OCCURN_FLG', 'ST_OCCURN_FLG', 'FIRE_OCCURN_FLG', 
             'FOUND_OPEN_OCCURN_FLG','PUBLIC_SAFETY_OCCURN_FLG', 'WIRE_OCCURN_FLG', 'SWITCH_OCCURN_FLG',
             'REGULATOR_OCCURN_FLG', 'CUTOUT_OCCURN_FLG','CAP_BANK_OCCURN_FLG','RECLOSER_OCCURN_FLG','OH_OCCURN_FLG',
             'PRIORITY_VAL_1.0','PRIORITY_VAL_2.0','PRIORITY_VAL_3.0','PRIORITY_VAL_5.0', 'CITY_NAM', 'PRIM_OCCURN_DESC', 
                 'SECO_OCCURN_DESC', 'TERT_OCCURN_DESC', 'PRIM_CAUSE_DESC', 'SECO_CAUSE_DESC','LAT', 'LONG', 'ZONE']]
df_ads.columns = ['OUTAGE_ID','INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 
         'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DOWNSTREAM_KVA_VAL', 
         'KVA_VAL', 'DAY_FLAG', 'TTR',
                  'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG','POWER_OUT_CLUE_FLG','OPEN_DEVICE_CLUE_FLG',
                'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG','IVR_CLUE_FLG','EQUIPMENT_CLUE_FLG','TRANSFORMER_CLUE_FLG',
             'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG','WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG',
             'PUBLIC_CAUSE_FLG','WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG','WEATHER__WIND_CAUSE_FLG',
             'WEATHER__HEAT_CAUSE_FLG','CUST_REQUEST_CAUSE_FLG','WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG',
             'SUBSTATION_CAUSE_FLG','TREE_CAUSE_FLG','MISCELLANEOUS_CAUSE_FLG','NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 
              'NO_OUTAGE_CAUSE_FLG',
             'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 
             'METER_OCCURN_FLG', 'SERVICE_OCCURN_FLG','CABLE_OCCURN_FLG', 'ST_OCCURN_FLG', 'FIRE_OCCURN_FLG', 
             'FOUND_OPEN_OCCURN_FLG','PUBLIC_SAFETY_OCCURN_FLG', 'WIRE_OCCURN_FLG', 'SWITCH_OCCURN_FLG',
             'REGULATOR_OCCURN_FLG', 'CUTOUT_OCCURN_FLG','CAP_BANK_OCCURN_FLG','RECLOSER_OCCURN_FLG','OH_OCCURN_FLG',
             'PRIORITY_VAL_1.0','PRIORITY_VAL_2.0','PRIORITY_VAL_3.0','PRIORITY_VAL_5.0', 'CITY_NAM','PRIM_OCCURN_DESC', 'SECO_OCCURN_DESC',
                  'TERT_OCCURN_DESC', 'PRIM_CAUSE_DESC', 'SECO_CAUSE_DESC', 'LAT', 'LONG','ZONE']

## LIVE OUTAGE

In [ ]:
df_live_out = df_ads[['OUTAGE_ID', 'CREATION_DATETIME', 'ENERGIZED_DATETIME']]

In [ ]:
def count_outage(group):
    group = group.reset_index(drop = True)
    group['LIVE_OUTAGE'] = len(df_live_out[(df_live_out.CREATION_DATETIME < group.CREATION_DATETIME[0]) & (df_live_out.ENERGIZED_DATETIME > group.CREATION_DATETIME[0])])
    return group

df_live_out = df_live_out.groupby(['OUTAGE_ID'], as_index = False).apply(count_outage)

In [ ]:
df_ads = pd.merge(df_ads, df_live_out[['OUTAGE_ID','LIVE_OUTAGE']], on = ['OUTAGE_ID'], how = "left")

In [ ]:
print(df_ads.shape)

(111163, 79)


In [ ]:
df_ads = df_ads[df_ads.EVENT=='STORM']
print(df_ads.shape)

(19673, 79)


### ACTIVE NUMBER OF CREWS

In [ ]:
# Read d16_memb_activity table from GCS landing bucket
d16_memb_activity = pd.read_csv(r'gs://aes-datahub-0001-landing/CAD/IPL/CAD.D16_MEMB_ACTIVITY.csv', sep = ";", encoding = "ISO-8859-1")
d16_memb_activity['Date'] = pd.to_datetime(d16_memb_activity['D16_DATETIME_KEY'].astype(str).str[:8])

#filter for DIST.OPS and DUTY.IN
d16_memb_activity = d16_memb_activity[(d16_memb_activity.AGENCY_CODE == 'DIST.OPS') & (d16_memb_activity.D16S_CAD_ACTION == 'DUTY-IN')]

active_crews = d16_memb_activity.groupby(['Date'],as_index=False).agg({'UNIT' : pd.Series.nunique})
del d16_memb_activity
df_ads['Date'] = df_ads['CREATION_DATETIME'].dt.date
df_ads['Date'] = pd.to_datetime(df_ads['Date'])
active_crews['Date'] = pd.to_datetime(active_crews['Date'])
print(df_ads['Date'].dtype)
print(active_crews['Date'].dtype)

df_ads = pd.merge(df_ads, active_crews, how='left', on=['Date'])
print(df_ads.shape)

datetime64[ns]
datetime64[ns]
(19673, 81)


## Dark Sky Weather Data Marker Mapping 

In [ ]:
list_columns = ['LAT','LONG']
df_ads[list_columns] = df_ads[list_columns].apply(pd.to_numeric, errors='coerce')

ads = df_ads[df_ads['EVENT']=='STORM']
ads=ads[['OUTAGE_ID', 'LAT', 'LONG']]
ads.reset_index(drop=True,inplace=True)
ads.head()

,OUTAGE_ID,LAT,LONG
0,2001317842,40,-86
1,2001317865,40,-86
2,2001317856,40,-87
3,2001317899,40,-86
4,2001317873,40,-86


In [ ]:
ads['Marker1_LAT'] =  39.9613 
ads['Marker2_LAT'] = 39.8971
ads['Marker3_LAT'] = 39.9060
ads['Marker4_LAT'] = 39.9024
ads['Marker5_LAT'] = 39.8960
ads['Marker6_LAT'] = 39.8339
ads['Marker7_LAT'] = 39.8412
ads['Marker8_LAT'] = 39.8381
ads['Marker9_LAT'] = 39.8386
ads['Marker10_LAT'] = 39.7579
ads['Marker11_LAT'] = 39.7621
ads['Marker12_LAT'] = 39.7621
ads['Marker13_LAT'] = 39.7695
ads['Marker14_LAT'] = 39.6617
ads['Marker15_LAT'] = 39.6639
ads['Marker16_LAT'] = 39.6702
ads['Marker17_LAT'] = 39.6744
ads['Marker18_LAT'] = 39.5909
ads['Marker19_LAT'] = 39.5295
ads['Marker20_LAT'] = 39.5475

ads['Marker1_LONG'] = -86.4034 
ads['Marker2_LONG'] = -86.3045
ads['Marker3_LONG'] = -86.2001
ads['Marker4_LONG'] = -86.0738
ads['Marker5_LONG'] = -85.9783
ads['Marker6_LONG'] = -86.3155
ads['Marker7_LONG'] = -86.2056
ads['Marker8_LONG'] = -86.0985
ads['Marker9_LONG'] = -85.9811
ads['Marker10_LONG'] = -86.3155
ads['Marker11_LONG'] = -86.2042
ads['Marker12_LONG'] = -86.0923
ads['Marker13_LONG'] = -85.9708
ads['Marker14_LONG'] = -86.2935
ads['Marker15_LONG'] = -86.1823
ads['Marker16_LONG'] = -86.0669
ads['Marker17_LONG'] = -85.9557
ads['Marker18_LONG'] = -86.4212
ads['Marker19_LONG'] = -86.5874
ads['Marker20_LONG'] = -86.2743


In [ ]:
# calculate distance from 2 lat long 
import math
def haversine(p1, p2):
    R = 6371     # earth radius in km
    p1 = [math.radians(v) for v in p1]
    p2 = [math.radians(v) for v in p2]

    d_lat = p2[0] - p1[0]
    d_lng = p2[1] - p1[1]
    a = math.pow(math.sin(d_lat / 2), 2) + math.cos(p1[0]) * math.cos(p2[0]) * math.pow(math.sin(d_lng / 2), 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c   # returns distance between p1 and p2 in km


In [ ]:
# calculate minimum distance

def minimum_distance(lat, long, marker1_lat, marker2_lat, marker3_lat, marker4_lat, marker5_lat, marker6_lat, marker7_lat, marker8_lat, marker9_lat, marker10_lat, marker11_lat,
                     marker12_lat, marker13_lat, marker14_lat, marker15_lat, marker16_lat, marker17_lat, marker18_lat, marker19_lat, marker20_lat, marker1_long, marker2_long,
                     marker3_long, marker4_long, marker5_long, marker6_long, marker7_long, marker8_long, marker9_long, marker10_long, marker11_long, marker12_long, marker13_long,
                     marker14_long, marker15_long, marker16_long, marker17_long, marker18_long, marker19_long, marker20_long):
    import operator
    
    dist1 = haversine((lat,long), (marker1_lat, marker1_long))
    dist2 = haversine((lat,long), (marker2_lat, marker2_long))
    dist3 = haversine((lat,long), (marker3_lat, marker3_long))
    dist4 = haversine((lat,long), (marker4_lat, marker4_long))
    dist5 = haversine((lat,long), (marker5_lat, marker5_long))
    dist6 = haversine((lat,long), (marker6_lat, marker6_long))
    dist7 = haversine((lat,long), (marker7_lat, marker7_long))
    dist8 = haversine((lat,long), (marker8_lat, marker8_long))
    dist9 = haversine((lat,long), (marker9_lat, marker9_long))
    dist10 = haversine((lat,long), (marker10_lat, marker10_long))
    dist11 = haversine((lat,long), (marker11_lat, marker11_long))
    dist12 = haversine((lat,long), (marker12_lat, marker12_long))
    dist13 = haversine((lat,long), (marker13_lat, marker13_long))
    dist14 = haversine((lat,long), (marker14_lat, marker14_long))
    dist15 = haversine((lat,long), (marker15_lat, marker15_long))
    dist16 = haversine((lat,long), (marker16_lat, marker16_long))
    dist17 = haversine((lat,long), (marker17_lat, marker17_long))
    dist18 = haversine((lat,long), (marker18_lat, marker18_long))
    dist19 = haversine((lat,long), (marker19_lat, marker19_long))
    dist20 = haversine((lat,long), (marker20_lat, marker20_long))
    
    dist_list = [dist1, dist2, dist3, dist4, dist5, dist6, dist7, dist8, dist9, dist10, dist11, dist12, dist13, dist14, dist15, dist16, dist17, dist18, dist19, dist20]

    min_index, min_value = min(enumerate(dist_list), key=operator.itemgetter(1))
    
    if ( (math.isnan(lat)) | (math.isnan(long)) ):
        return None, None
    else :
        return min_value, min_index+1

In [ ]:
ads['Min_Distance'], ads['Marker_Location'] = zip(*ads.apply(lambda row: minimum_distance(row['LAT'], row['LONG'], row['Marker1_LAT'], row['Marker2_LAT'],
                                                            row['Marker3_LAT'], row['Marker4_LAT'], row['Marker5_LAT'], row['Marker6_LAT'],
                                                            row['Marker7_LAT'], row['Marker8_LAT'], row['Marker9_LAT'], row['Marker10_LAT'], 
                                                            row['Marker11_LAT'], row['Marker12_LAT'], row['Marker13_LAT'], row['Marker14_LAT'],
                                                            row['Marker15_LAT'], row['Marker16_LAT'], row['Marker17_LAT'], row['Marker18_LAT'],
                                                            row['Marker19_LAT'], row['Marker20_LAT'], row['Marker1_LONG'], row['Marker2_LONG'],
                                                            row['Marker3_LONG'], row['Marker4_LONG'], row['Marker5_LONG'], row['Marker6_LONG'], 
                                                            row['Marker7_LONG'], row['Marker8_LONG'], row['Marker9_LONG'], row['Marker10_LONG'],
                                                            row['Marker11_LONG'], row['Marker12_LONG'], row['Marker13_LONG'], row['Marker14_LONG'],
                                                            row['Marker15_LONG'], row['Marker16_LONG'], row['Marker17_LONG'], row['Marker18_LONG'], 
                                                            row['Marker19_LONG'], row['Marker20_LONG']),axis=1))

In [ ]:
ads = ads[['OUTAGE_ID', 'LAT', 'LONG', 'Min_Distance', 'Marker_Location']]
ads.head()

,OUTAGE_ID,LAT,LONG,Min_Distance,Marker_Location
0,2001317842,40,-86,6,12
1,2001317865,40,-86,5,3
2,2001317856,40,-87,2,19
3,2001317899,40,-86,5,11
4,2001317873,40,-86,6,18


In [ ]:
marker_count = ads.groupby(['Marker_Location'],as_index=False).agg({'OUTAGE_ID' : 'count'})

In [ ]:
marker_count.sort_values(['OUTAGE_ID'],ascending=False,inplace=True)
print(ads.shape[0])
print(sum(marker_count.OUTAGE_ID))
print(ads.LAT.isnull().sum())
print(ads.shape[0] - sum(marker_count.OUTAGE_ID))
marker_count

19673
19514
159
159


,Marker_Location,OUTAGE_ID
7,8,2999
6,7,1958
11,12,1732
3,4,1704
10,11,1679
2,3,1638
14,15,1306
8,9,1005
15,16,908
9,10,765


In [ ]:
ads['Marker_Location'] = 'Marker' + ads['Marker_Location'].astype(str)
print(ads.Marker_Location.unique())

['Marker12' 'Marker3' 'Marker19' 'Marker11' 'Marker18' 'Marker9' 'Marker5'
 'Marker16' 'Marker10' 'Marker15' 'Marker14' 'Marker7' 'Marker2' 'Marker4'
 'Marker8' 'Marker13' 'Marker1' 'Marker6' 'Marker17' 'Marker20'
 'MarkerNone']


In [ ]:
marker1=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker1_storm_data.csv')
marker2=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker2_storm_data.csv')
marker3=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker3_storm_data.csv')
marker4=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker4_storm_data.csv')
marker5=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker5_storm_data.csv')
marker6=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker6_storm_data.csv')
marker7=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker7_storm_data.csv')
marker8=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker8_storm_data.csv')
marker9=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker9_storm_data.csv')
marker10=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker10_storm_data.csv')
marker11=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker11_storm_data.csv')
marker12=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker12_storm_data.csv')
marker13=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker13_storm_data.csv')
marker14=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker14_storm_data.csv')
marker15=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker15_storm_data.csv')
marker16=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker16_storm_data.csv')
marker17=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker17_storm_data.csv')
marker18=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker18_storm_data.csv')
marker19=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker19_storm_data.csv')
marker20=pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/Marker20_storm_data.csv')

In [ ]:
marker_final=pd.concat([marker1,
marker2,
marker3,
marker4,
marker5,
marker6,
marker7,
marker8,
marker9,
marker10,
marker11,
marker12,
marker13,
marker14,
marker15,
marker16,
marker17,
marker18,
marker19,
marker20],ignore_index=True)

In [ ]:
marker_final=marker_final[['Date', 'Location', 'cloudCover', 'dewPoint', 'humidity', 'icon', 'nearest-station',
                           'precipIntensity', 'precipIntensityMax', 'precipType', 'pressure', 'temperatureMax', 
                           'temperatureMin', 'visibility', 'windBearing', 'windGust', 'windSpeed']]
print(marker_final.shape)

(8500, 17)


In [ ]:
marker_final.to_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/All_Markers_Combined_Storm.csv')

In [ ]:
marker_final['Date']=pd.to_datetime(marker_final['Date']).dt.date

In [ ]:
ads.drop(['LAT','LONG'],axis=1,inplace=True)
ads.head()
print(df_ads.shape)
df_ads = pd.merge(df_ads, ads, how='left', on=['OUTAGE_ID'])
print(df_ads.shape)

(19673, 81)
(19673, 83)


In [ ]:
print(df_ads['Date'].dtype)
marker_final['Date'] = pd.to_datetime(marker_final['Date'])
print(marker_final['Date'].dtype)
ads_final=df_ads.merge(marker_final,how='left',left_on=['Date','Marker_Location'],right_on=['Date','Location'])
ads_final.drop(['Date','Location'],axis=1,inplace=True)
print(ads_final.shape)

datetime64[ns]
datetime64[ns]
(19673, 97)


In [ ]:
ads_final.rename(columns = {'cloudCover' : 'CLOUDCOVER',
                            'dewPoint' : 'DEWPOINT',
                            'humidity' : 'HUMIDITY',
                            'icon' : 'ICON',
                            'nearest-station' : 'NEAREST-STATION',
                            'precipIntensity' : 'PRECIPINTENSITY',
                            'precipIntensityMax' : 'PRECIPINTENSITYMAX',
                            'precipType' : 'PRECIPTYPE',
                            'pressure' : 'PRESSURE',
                            'temperatureMax' : 'TEMPERATUREMAX',
                            'temperatureMin' : 'TEMPERATUREMIN',
                            'visibility' : 'VISIBILITY',
                            'windBearing' : 'WINDBEARING',
                            'windGust' : 'WINDGUST',
                            'windSpeed' : 'WINDSPEED'}, inplace=True)

In [ ]:
print(list(ads_final.columns))

['OUTAGE_ID', 'INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DOWNSTREAM_KVA_VAL', 'KVA_VAL', 'DAY_FLAG', 'TTR', 'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG', 'POWER_OUT_CLUE_FLG', 'OPEN_DEVICE_CLUE_FLG', 'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG', 'IVR_CLUE_FLG', 'EQUIPMENT_CLUE_FLG', 'TRANSFORMER_CLUE_FLG', 'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG', 'WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG', 'PUBLIC_CAUSE_FLG', 'WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG', 'WEATHER__WIND_CAUSE_FLG', 'WEATHER__HEAT_CAUSE_FLG', 'CUST_REQUEST_CAUSE_FLG', 'WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG', 'SUBSTATION_CAUSE_FLG', 'TREE_CAUSE_FLG', 'MISCELLANEOUS_CAUSE_FLG', 'NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 'NO_OUTAGE_CAUSE_FLG', 'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 

## Merge Cluster Profiles

In [ ]:
clusters = pd.read_csv("CLuster_profiles_darksky_storm_k-prototypes.csv")
clusters.drop(clusters.columns[clusters.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [ ]:
clusters = clusters[['Date','Location','clusters']]
clusters['Date']=pd.to_datetime(clusters['Date']).dt.date

In [ ]:
ads_final['Date'] = ads_final['CREATION_DATETIME'].dt.date
ads_final = ads_final.merge(clusters,how='left',left_on=['Date','Marker_Location'],right_on=['Date','Location'])
ads_final['clusters'] = 'Cluster' + ads_final['clusters'].astype(str)

In [ ]:
ads_final.drop(['Date','Location'],axis=1,inplace=True)
print(ads_final.shape)

(19673, 98)


In [ ]:
print(list(ads_final.columns))

['OUTAGE_ID', 'INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DOWNSTREAM_KVA_VAL', 'KVA_VAL', 'DAY_FLAG', 'TTR', 'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG', 'POWER_OUT_CLUE_FLG', 'OPEN_DEVICE_CLUE_FLG', 'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG', 'IVR_CLUE_FLG', 'EQUIPMENT_CLUE_FLG', 'TRANSFORMER_CLUE_FLG', 'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG', 'WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG', 'PUBLIC_CAUSE_FLG', 'WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG', 'WEATHER__WIND_CAUSE_FLG', 'WEATHER__HEAT_CAUSE_FLG', 'CUST_REQUEST_CAUSE_FLG', 'WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG', 'SUBSTATION_CAUSE_FLG', 'TREE_CAUSE_FLG', 'MISCELLANEOUS_CAUSE_FLG', 'NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 'NO_OUTAGE_CAUSE_FLG', 'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 

## ADD CYCLICITY ACCORDING TO HOUR

In [ ]:
ads_final['Hour'] = ads_final['CREATION_DATETIME'].dt.hour
print(ads_final['Hour'].unique())

[20 21 22 15 16 17 19 18 23  0  3  6  8 11 12  1  2  4  5  7  9 10 13 14]


In [ ]:
ads_final['Hour_Sin'] = np.sin(ads_final.Hour*(2.*np.pi/24))
ads_final['Hour_Cos'] = np.cos(ads_final.Hour*(2.*np.pi/24))

In [ ]:
ads_final.loc[ads_final['ZONE'] == '1', 'ZONE'] = 'ZONE1'
ads_final.loc[ads_final['ZONE'] == '2', 'ZONE'] = 'ZONE2'
ads_final.loc[ads_final['ZONE'] == '3', 'ZONE'] = 'ZONE3'
ads_final.loc[ads_final['ZONE'] == '4', 'ZONE'] = 'ZONE4'

In [ ]:
ads_final.drop(['Hour'],axis=1,inplace=True)

## MAJ_OTG_ID ADDTION

In [ ]:
ads_final = pd.merge(ads_final, maj_otg_df, on=['INCIDENT_ID', 'STRCTUR_NO'], how='left')
ads_final.head()

,OUTAGE_ID,INCIDENT_ID,STRCTUR_NO,EVENT,CREATION_DATETIME,ENERGIZED_DATETIME,CIRCT_ID,DNI_EQUIP_TYPE,SUBST_ID,CALL_QTY,DOWNSTREAM_CUST_QTY,KEY_CUST_QTY,ETR_DATETIME,CUST_QTY,DOWNSTREAM_KVA_VAL,KVA_VAL,DAY_FLAG,TTR,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,PRIM_OCCURN_DESC,SECO_OCCURN_DESC,TERT_OCCURN_DESC,PRIM_CAUSE_DESC,SECO_CAUSE_DESC,LAT,LONG,ZONE,LIVE_OUTAGE,UNIT,Min_Distance,Marker_Location,CLOUDCOVER,DEWPOINT,HUMIDITY,ICON,NEAREST-STATION,PRECIPINTENSITY,PRECIPINTENSITYMAX,PRECIPTYPE,PRESSURE,TEMPERATUREMAX,TEMPERATUREMIN,VISIBILITY,WINDBEARING,WINDGUST,WINDSPEED,clusters,Hour_Sin,Hour_Cos,MAJ_OTG_ID
0,2001317842,2000391296,648-B/72,STORM,2007-02-24 20:02:00,2007-02-24 21:45:00,1702,ASWITCH,170,183,1282,4,2007-02-24 21:32:00,1282,6101,6101,0,103,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,SUBSTATION,LOCK OUT,None,MISCELLANEOUS,NO CAUSE-PATROLLED,40,-86,2,8,nan,6,Marker12,1,19,1,snow,4,0,0,snow,1019,35,22,9,102,28,8,Cluster3.0,-1,0,20070001001
1,2001317865,2000391302,237--/42,STORM,2007-02-24 20:18:26,2007-02-24 21:50:00,1402,FUSE,140,2,4,0,2007-02-24 22:48:26,4,100,100,0,92,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,FUSE,FUSE BLOWN,None,TREE,ON PRIMARY,40,-86,3,8,nan,5,Marker3,1,18,1,snow,2,0,0,snow,1019,35,23,9,103,27,7,Cluster3.0,-1,0,20070001001
2,2001317856,2000391313,M15-U/168,STORM,2007-02-24 20:31:55,2007-02-25 00:23:42,7001,1TBOH,700,1,1,0,2007-02-25 00:01:55,1,15,15,0,232,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,MONROVIA,FUSE,DEFECTIVE FUSE,None,MISCELLANEOUS,OTHER,40,-87,1,9,nan,2,Marker19,1,21,1,rain,11,0,0,rain,1018,38,24,9,101,24,8,Cluster7.0,-1,0,20070001001
3,2001317899,2000391321,495-A/106,STORM,2007-02-24 20:56:03,2007-02-25 00:30:00,1257,FUSE,125,133,271,2,2007-02-24 23:26:03,271,504,504,0,214,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,INDIANAPOLIS,FUSE,FUSE BLOWN,None,O.H. EQUIPMENT,BAD OR BROKEN INSULATOR,40,-86,4,10,nan,5,Marker11,1,19,1,snow,5,0,0,snow,1019,36,22,9,104,30,10,Cluster3.0,-1,0,20070001001
4,2001317873,2000391329,801QA/2,STORM,2007-02-24 20:59:10,2007-02-24 21:15:02,7056,FUSE,705,4,70,1,2007-02-24 23:29:10,70,870,870,0,16,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,MOORESVILLE,POLE BURNING,POLE BURNING,None,O.H. EQUIPMENT,BAD CUTOUT OR BARREL,40,-86,1,11,nan,6,Marker18,1,19,1,snow,12,0,0,snow,1019,36,22,9,104,27,9,Cluster3.0,-1,0,20070001001


In [ ]:
print(list(ads_final.MAJ_OTG_ID.unique()))

[20070001001.0, 20070002001.0, 20070003001.0, 20070004001.0, 20070005001.0, 20070006001.0, 20070007002.0, 20070008001.0, 20070009001.0, 20070010001.0, 20070011001.0, 20070012001.0, 20070013001.0, 20080001002.0, 20080002001.0, 20080003001.0, 20080004002.0, 20080005001.0, 20080006003.0, 20080007002.0, 20080008003.0, 20080009001.0, 20080010001.0, 20080011001.0, 20080012002.0, 20080013002.0, 20080014001.0, 20080015001.0, 20080016001.0, 20080017001.0, 20080018001.0, 20080019003.0, 20080020001.0, 20080021001.0, 20080022001.0, 20080023001.0, 20080024001.0, 20090001002.0, 20090002002.0, 20090003001.0, 20090004002.0, 20090005002.0, 20090006001.0, 20090007002.0, 20090008001.0, 20090009002.0, 20090010001.0, 20090011003.0, 20090012001.0, 20090013001.0, 20090014001.0, 20100001001.0, 20100002001.0, 20100003001.0, 20100004001.0, 20100005001.0, 20100006001.0, 20100007001.0, 20100008001.0, 20100009002.0, 20100010001.0, 20100011001.0, 20100012002.0, 20100013001.0, 20100014001.0, 20100015001.0, 201000160

## STORM START STORM END ADDITION

In [ ]:
merge = ads_final.groupby(['MAJ_OTG_ID'],as_index=False).agg({'CREATION_DATETIME' :{ 'CREATION_DATETIME_MIN' : 'min', 'CREATION_DATETIME_MAX' : 'max'}})
merge.columns = merge.columns.get_level_values(1)
merge.rename(columns={'CREATION_DATETIME_MIN' : 'STORM_START', 'CREATION_DATETIME_MAX' : 'STORM_END', '' : 'MAJ_OTG_ID',}, inplace=True)

In [ ]:
ads_final = pd.merge(ads_final, merge, right_on = ['MAJ_OTG_ID'], left_on = ['MAJ_OTG_ID'], how='left')

In [ ]:
ads_final.head()

,OUTAGE_ID,INCIDENT_ID,STRCTUR_NO,EVENT,CREATION_DATETIME,ENERGIZED_DATETIME,CIRCT_ID,DNI_EQUIP_TYPE,SUBST_ID,CALL_QTY,DOWNSTREAM_CUST_QTY,KEY_CUST_QTY,ETR_DATETIME,CUST_QTY,DOWNSTREAM_KVA_VAL,KVA_VAL,DAY_FLAG,TTR,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,PRIM_OCCURN_DESC,SECO_OCCURN_DESC,TERT_OCCURN_DESC,PRIM_CAUSE_DESC,SECO_CAUSE_DESC,LAT,LONG,ZONE,LIVE_OUTAGE,UNIT,Min_Distance,Marker_Location,CLOUDCOVER,DEWPOINT,HUMIDITY,ICON,NEAREST-STATION,PRECIPINTENSITY,PRECIPINTENSITYMAX,PRECIPTYPE,PRESSURE,TEMPERATUREMAX,TEMPERATUREMIN,VISIBILITY,WINDBEARING,WINDGUST,WINDSPEED,clusters,Hour_Sin,Hour_Cos,MAJ_OTG_ID,STORM_END,STORM_START
0,2001317842,2000391296,648-B/72,STORM,2007-02-24 20:02:00,2007-02-24 21:45:00,1702,ASWITCH,170,183,1282,4,2007-02-24 21:32:00,1282,6101,6101,0,103,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,SUBSTATION,LOCK OUT,None,MISCELLANEOUS,NO CAUSE-PATROLLED,40,-86,2,8,nan,6,Marker12,1,19,1,snow,4,0,0,snow,1019,35,22,9,102,28,8,Cluster3.0,-1,0,20070001001,2007-02-24 22:49:02,2007-02-24 20:02:00
1,2001317865,2000391302,237--/42,STORM,2007-02-24 20:18:26,2007-02-24 21:50:00,1402,FUSE,140,2,4,0,2007-02-24 22:48:26,4,100,100,0,92,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,FUSE,FUSE BLOWN,None,TREE,ON PRIMARY,40,-86,3,8,nan,5,Marker3,1,18,1,snow,2,0,0,snow,1019,35,23,9,103,27,7,Cluster3.0,-1,0,20070001001,2007-02-24 22:49:02,2007-02-24 20:02:00
2,2001317856,2000391313,M15-U/168,STORM,2007-02-24 20:31:55,2007-02-25 00:23:42,7001,1TBOH,700,1,1,0,2007-02-25 00:01:55,1,15,15,0,232,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,MONROVIA,FUSE,DEFECTIVE FUSE,None,MISCELLANEOUS,OTHER,40,-87,1,9,nan,2,Marker19,1,21,1,rain,11,0,0,rain,1018,38,24,9,101,24,8,Cluster7.0,-1,0,20070001001,2007-02-24 22:49:02,2007-02-24 20:02:00
3,2001317899,2000391321,495-A/106,STORM,2007-02-24 20:56:03,2007-02-25 00:30:00,1257,FUSE,125,133,271,2,2007-02-24 23:26:03,271,504,504,0,214,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,INDIANAPOLIS,FUSE,FUSE BLOWN,None,O.H. EQUIPMENT,BAD OR BROKEN INSULATOR,40,-86,4,10,nan,5,Marker11,1,19,1,snow,5,0,0,snow,1019,36,22,9,104,30,10,Cluster3.0,-1,0,20070001001,2007-02-24 22:49:02,2007-02-24 20:02:00
4,2001317873,2000391329,801QA/2,STORM,2007-02-24 20:59:10,2007-02-24 21:15:02,7056,FUSE,705,4,70,1,2007-02-24 23:29:10,70,870,870,0,16,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,MOORESVILLE,POLE BURNING,POLE BURNING,None,O.H. EQUIPMENT,BAD CUTOUT OR BARREL,40,-86,1,11,nan,6,Marker18,1,19,1,snow,12,0,0,snow,1019,36,22,9,104,27,9,Cluster3.0,-1,0,20070001001,2007-02-24 22:49:02,2007-02-24 20:02:00


## ADD SUBSEQUENT OUTAGES 

In [ ]:
ads_final['RANK_SUBSEQUENT_MAJ_OTG_ID'] = ads_final.groupby(['MAJ_OTG_ID'])['CREATION_DATETIME'].rank(method='dense', ascending=True)

In [ ]:
print(list(ads_final.columns))
print(ads_final.shape)

['OUTAGE_ID', 'INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DOWNSTREAM_KVA_VAL', 'KVA_VAL', 'DAY_FLAG', 'TTR', 'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG', 'POWER_OUT_CLUE_FLG', 'OPEN_DEVICE_CLUE_FLG', 'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG', 'IVR_CLUE_FLG', 'EQUIPMENT_CLUE_FLG', 'TRANSFORMER_CLUE_FLG', 'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG', 'WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG', 'PUBLIC_CAUSE_FLG', 'WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG', 'WEATHER__WIND_CAUSE_FLG', 'WEATHER__HEAT_CAUSE_FLG', 'CUST_REQUEST_CAUSE_FLG', 'WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG', 'SUBSTATION_CAUSE_FLG', 'TREE_CAUSE_FLG', 'MISCELLANEOUS_CAUSE_FLG', 'NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 'NO_OUTAGE_CAUSE_FLG', 'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 

## Adding New Clusters

In [ ]:
df_clusters_new=pd.read_csv('clusters_seasonality_included.csv')
ads_final['Date']=pd.to_datetime(ads_final['CREATION_DATETIME']).dt.date
df_clusters_new['Date']=pd.to_datetime(df_clusters['Date']).dt.date

NameError: name 'df_clusters' is not defined

In [ ]:
df_clusters_new=df_clusters_new[[
       'Location', 'Date', 'clusters']]

In [ ]:
df_clusters_new.rename(columns={'Location':'Marker_Location','clusters':'cluters_new'},inplace=True)

In [ ]:
ads_final=ads_final.merge(df_clusters_new,how='left',left_on=['Date','Marker_Location'],right_on=['Date','Marker_Location'])

In [ ]:
ads_final['clusters_new']='Cluster'+ads_final['clusters_new'].as_type(str)

In [ ]:
ads_final['WIND_DIRECTION']='NONE'
ads_final['WIND_DIRECTION'][(ads_final['WINDBEARING']>=0)&(ads_final['WINDBEARING']<45)]='N-E-N'
ads_final['WIND_DIRECTION'][(ads_final['WINDBEARING']>=45)&(ads_final['WINDBEARING']<90)]='N-E-E'
ads_final['WIND_DIRECTION'][(ads_final['WINDBEARING']>=90)&(ads_final['WINDBEARING']<135)]='S-E-E'
ads_final['WIND_DIRECTION'][(ads_final['WINDBEARING']>=135)&(ads_final['WINDBEARING']<180)]='S-E-S'
ads_final['WIND_DIRECTION'][(ads_final['WINDBEARING']>=180)&(ads_final['WINDBEARING']<225)]='S-W-S'
ads_final['WIND_DIRECTION'][(ads_final['WINDBEARING']>=225)&(ads_final['WINDBEARING']<270)]='S-W-W'
ads_final['WIND_DIRECTION'][(ads_final['WINDBEARING']>=270)&(ads_final['WINDBEARING']<315)]='N-W-W'
ads_final['WIND_DIRECTION'][(ads_final['WINDBEARING']>=315)&(ads_final['WINDBEARING']<360)]='N-W-N'

In [ ]:
ads_final['MONTH']=pd.to_datetime(ads_final['CREATION_DATETIME']).dt.month
ads_final['SEASON']='NONE'
ads_final['SEASON'][((ads_final['MONTH']>=1)&(ads_final['MONTH']<=3))|(ads_final['MONTH']==12)]='WINTER'
ads_final['SEASON'][(ads_final['MONTH']>=4)&(ads_final['MONTH']<=6)]='SPRING'
ads_final['SEASON'][(ads_final['MONTH']>=7)&(ads_final['MONTH']<=9)]='SUMMER'
ads_final['SEASON'][(ads_final['MONTH']>=10)&(ads_final['MONTH']<=11)]='FALL'

In [ ]:
df_clusters=df_clusters.drop(['Date','MONTH'],axis=1)

In [ ]:
df_clusters.head()

## WRITE TO CSV 

In [ ]:
df_clusters.to_csv("gs://aes-datahub-0001-curated/Outage_Restoration/Historical_Data/Master_Dataset/OMS_IPL_OUTAGE_DATASET_V8.5.csv")